In [79]:
import pandas as pd

df = pd.read_csv('../dataset.csv')
df = df.head(10)
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head(10)


,url,title,description,cat1,cat2,cat3
0,https://www.tourisme-cambresis.fr/1-les-templi...,"Aventure-jeu : ""Les Templiers du coffre d'or""",Le jeu aventure « Les templiers du coffre d’or...,Jeu,Famille,Détente
1,https://www.tourisme-cambresis.fr/exposition-t...,"Exposition ""le tank dans tous ses états""","L'apparition des tanks, arme de guerre incarna...",Exposition,Culture,NaN
2,https://www.tourisme-cambresis.fr/femme-premie...,"Exposition ""La femme au premier millénaire""",L’exposition vous invite à une plongée au cœur...,Exposition,Culture,NaN
3,https://www.tourisme-cambresis.fr/exposition-a...,"Exposition ""L'archéologie à Vaucelles, une his...","L’abbaye de Vaucelles, monument culturel du Dé...",Exposition,Culture,Histoire
4,https://www.tourisme-cambresis.fr/exposition-e...,"Exposition ""Les chasseurs cyclistes au combat ...","Exposition ""Les chasseurs cyclistes au combat ...",Exposition,Histoire,Culture
5,https://www.tourisme-cambresis.fr/exposition-c...,"Exposition collective d'art actuel ""Transfo""","Avec « Transfo », Modulo présente sa premièr...",Exposition,Art,Culture
6,https://www.tourisme-cambresis.fr/exposition-d...,"Exposition ""S'est-on déjà brûlé ?""","L'exposition ""S'est-on déjà brûlé ?"" s'inscrit...",Exposition,Art,Culture
7,https://www.tourisme-cambresis.fr/2-jardin-fet...,Jardin en fête,Jardin en fête\n\nParc de loisirs familial sit...,Détente,Famille,Santé
8,https://www.tourisme-cambresis.fr/decryptex-ma...,DECRYPTEX - la machine de la chapelle,"""DECRYPTEX - La machine de la chapelle""\n\n \n...",Jeu,Famille,Environnement
9,https://www.tourisme-cambresis.fr/visite-libre...,Visite libre de la Chapelle des Jésuites,Visitez librement la chapelle librement ou ave...,Visite,Culture,NaN


In [80]:
def extract_domain(url):
    url = url[url.find('//')+2:]
    url = url[:url.find('/')]
    return url

df['domain'] = df['url'].apply(extract_domain)

print("nb ligne df", len(df))

df = df.dropna(subset=['description'])

print("nb ligne df1", len(df))

df1 = df.copy()
df2 = df.dropna(subset=['cat2'])
df3 = df.dropna(subset=['cat3'])

print("nb ligne df2", len(df2))
print("nb ligne df3", len(df3))

df[['domain', 'url']].head(1)


nb ligne df 10
nb ligne df1 10
nb ligne df2 10
nb ligne df3 7


,domain,url
0,www.tourisme-cambresis.fr,https://www.tourisme-cambresis.fr/1-les-templi...


In [81]:
# générateur de tokens
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("camembert-base")


In [82]:
# fonction de prétraitement des tokens pour les tronqués pour par qu'ils dépassent la longueur max d'entrée du modèle
def preprocess_function(examples):
    return tokenizer(examples["description"], truncation=True)


In [83]:
# application de la fonction avec un accélérateur de mapping
# tokenized_df1 = df1.map(preprocess_function, batched=True)
tokenized_df1 = df1.apply(preprocess_function, axis=1)


In [84]:
from sklearn.model_selection import train_test_split

small_train_dataset, small_eval_dataset = train_test_split(tokenized_df1, test_size=0.2, random_state=42)

print("small_train_dataset:", small_train_dataset.shape)
print("small_eval_dataset:", small_eval_dataset.shape)


small_train_dataset: (8,)
small_eval_dataset: (2,)


In [85]:
# Now create a batch of examples using DataCollatorWithPadding. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [86]:
# création de IDs pour les labels
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}


In [87]:
# entraienement avec DistilBERT
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [88]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)


In [89]:
#  avec fonction évaluer les prédictions
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


In [90]:
# métrics
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


In [91]:
# monitoring
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    push_to_hub=True,
)


In [92]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 